# 03 pQTL deCODE processing

**Origin:** `0_3_processing_pqtl_decode.ipynb`  
**This annotated version was generated on:** 2025-10-13 06:41

**What this notebook does (high level):**  
- Process deCODE pQTL summary data: harmonize identifiers and prepare for cross-cohort meta-analysis.

**How to use:**  
1. Review the markdown notes before each code cell.  
2. Adjust input/output paths as needed for your environment.  
3. Run cell-by-cell to reproduce artifacts for downstream steps.

---


**Step 1:** Load tabular data (summary stats / annotations).

In [10]:
import os
import pandas as pd
import gzip
from pybedtools import BedTool

# Paths
pqtl_dir = "/mnt/f/0.data/decode/pqtl_2021/assocs_filtered/"
dhs_path = "/mnt/f/0.data/dhs/DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz"
output_dir = "/mnt/f/10_osteo_MR/datasets/dhs_context/pqtl_decode"

os.makedirs(output_dir, exist_ok=True)

# 1. Load and filter DHS index
dhs = pd.read_csv(dhs_path, sep='\t')
mask = dhs['component'].isin(['Lymphoid', 'Myeloid / erythroid'])
dhs_sub = dhs[mask].copy()

# Prepare DHS as BedTool
dhs_bed = BedTool.from_dataframe(
    dhs_sub[['seqname', 'start', 'end']].rename(
        columns={'seqname': 'chrom'}
    )
)

# 2. Processing function
def process_pqtl_file(pqtl_file, dhs_bed, output_dir):
    # Load pQTL data
    if pqtl_file.endswith('.gz'):
        pqtl_df = pd.read_csv(pqtl_file, sep='\t', compression='gzip')
    else:
        pqtl_df = pd.read_csv(pqtl_file, sep='\t')
    # Filter pQTL by significance
    pqtl_df = pqtl_df[pqtl_df['Pval'].astype(float) < 5e-8].copy()
    print( pqtl_df.shape )
    if pqtl_df.empty:
        return
    # Prepare BED for overlap
    pqtl_bed = BedTool.from_dataframe(
        pqtl_df.assign(
            chrom=pqtl_df['Chrom'] , # lambda df: 'chr' + df['Chrom'].astype(str),
            start=pqtl_df['Pos'] - 1, # BED is zero-based, half-open
            end=pqtl_df['Pos']
        )[['chrom', 'start', 'end', 'Name']]
    )
    # Intersect with DHS
    overlap = pqtl_bed.intersect(dhs_bed, wa=True, u=True)
    overlap_names = set([x.name for x in overlap])
    pqtl_filtered = pqtl_df[pqtl_df['Name'].isin(overlap_names)]
    
    if not pqtl_filtered.empty:
        # Write output
        basename = os.path.basename(pqtl_file).replace('.txt.gz', '_dhs.txt.gz').replace('.txt', '_dhs.txt.gz')
        pqtl_filtered.to_csv(os.path.join(output_dir, basename), sep='\t', index=False, compression='gzip')


# 3. Batch process all pQTL files
for fname in os.listdir(pqtl_dir):
    print( fname )
    if not fname.endswith('.txt.gz'):
        continue
    pqtl_file = os.path.join(pqtl_dir, fname)
    process_pqtl_file(pqtl_file, dhs_bed, output_dir)

/tmp/ipykernel_68782/2863000700.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dhs = pd.read_csv(dhs_path, sep='\t')


**Step 2:** Join/merge datasets to align keys across resources.

In [11]:

# 3. Batch process all pQTL files
for fname in os.listdir(pqtl_dir):
    print( fname )
    if not fname.endswith('.txt.gz'):
        continue
    pqtl_file = os.path.join(pqtl_dir, fname)
    process_pqtl_file(pqtl_file, dhs_bed, output_dir)


10036_201_ZHX3_ZHX3.txt.gz
(4897, 12)
10036_201_ZHX3_ZHX3.txt.md5sum
10037_98_SIGLEC12_SIG12.txt.gz
(4717, 12)
10037_98_SIGLEC12_SIG12.txt.md5sum
10040_63_TP63_P73L.txt.gz
(1, 12)
10040_63_TP63_P73L.txt.md5sum
10041_3_HNF4A_HNF4A.txt.gz
(12611, 12)
10041_3_HNF4A_HNF4A.txt.md5sum
10042_8_SGK3_SGK3.txt.gz
(107, 12)
10042_8_SGK3_SGK3.txt.md5sum
10043_31_BRD4_BRD4.txt.gz
(269, 12)
10043_31_BRD4_BRD4.txt.md5sum
10044_12_WNT10A_WN10A.txt.gz
(5225, 12)
10044_12_WNT10A_WN10A.txt.md5sum
10045_47_CUL3_CUL3.txt.gz
(80, 12)
10045_47_CUL3_CUL3.txt.md5sum
10046_55_BIRC2_cIAP_1.txt.gz
(2639, 12)
10046_55_BIRC2_cIAP_1.txt.md5sum
10047_12_NCF2_NCF_2.txt.gz
(3765, 12)
10047_12_NCF2_NCF_2.txt.md5sum
10048_7_CBFB_PEBB.txt.gz
(1239, 12)
10048_7_CBFB_PEBB.txt.md5sum
10049_112_TERF1_TERF1.txt.gz
(154, 12)
10049_112_TERF1_TERF1.txt.md5sum
10053_5_ILK_ILK1.txt.gz
(364, 12)
10053_5_ILK_ILK1.txt.md5sum
10054_3_GAN_GAN.txt.gz
(4, 12)
10054_3_GAN_GAN.txt.md5sum
10056_5_FOXM1_FOXM1.txt.gz
(301, 12)
10056_5_FOXM1_FO

KeyboardInterrupt: 

**Step 3:** Run a processing or analysis step.

In [45]:
qtlfiles = os.listdir(pqtl_dir)[ 734: ]


qtlfiles

['11355_10_EIF5A2_IF5A2.txt.gz',
 '11355_10_EIF5A2_IF5A2.txt.md5sum',
 '11356_19_DGCR14_DGC14.txt.gz',
 '11356_19_DGCR14_DGC14.txt.md5sum',
 '11358_15_GRB10_GRB10.txt.gz',
 '11358_15_GRB10_GRB10.txt.md5sum',
 '11360_39_RRM1_RRM1.txt.gz',
 '11360_39_RRM1_RRM1.txt.md5sum',
 '11361_73_TYMP_TP.txt.gz',
 '11361_73_TYMP_TP.txt.md5sum',
 '11363_58_AQP4_AQP4.txt.gz',
 '11363_58_AQP4_AQP4.txt.md5sum',
 '11364_18_MFN1_MFN1.txt.gz',
 '11364_18_MFN1_MFN1.txt.md5sum',
 '11365_17_TENM4_TEN4.txt.gz',
 '11365_17_TENM4_TEN4.txt.md5sum',
 '11368_32_AK2_KAD2.txt.gz',
 '11368_32_AK2_KAD2.txt.md5sum',
 '11369_23_ADH5_ADHX.txt.gz',
 '11369_23_ADH5_ADHX.txt.md5sum',
 '11370_20_INPP5E_INP5E.txt.gz',
 '11370_20_INPP5E_INP5E.txt.md5sum',
 '11371_1_GPR101_GP101.txt.gz',
 '11371_1_GPR101_GP101.txt.md5sum',
 '11372_2_ZNF18_ZNF18.txt.gz',
 '11372_2_ZNF18_ZNF18.txt.md5sum',
 '11375_49_FOXL2_FOXL2.txt.gz',
 '11375_49_FOXL2_FOXL2.txt.md5sum',
 '11377_19_ADH7_ADH7.txt.gz',
 '11377_19_ADH7_ADH7.txt.md5sum',
 '11378_37_S

**Step 4:** Load tabular data (summary stats / annotations).

In [46]:
import os
import pandas as pd
from pybedtools import BedTool
import concurrent.futures

# Paths
pqtl_dir = "/mnt/f/0.data/decode/pqtl_2021/assocs_filtered/"
dhs_path = "/mnt/f/0.data/dhs/DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz"
output_dir = "/mnt/f/10_osteo_MR/datasets/dhs_context/pqtl_decode"
os.makedirs(output_dir, exist_ok=True)

# 1. Load and filter DHS index, prepare BedTool as global
dhs = pd.read_csv(dhs_path, sep='\t')
mask = dhs['component'].isin(['Lymphoid', 'Myeloid / erythroid'])
dhs_sub = dhs[mask].copy()
dhs_bed = BedTool.from_dataframe(
    dhs_sub[['seqname', 'start', 'end']].rename(columns={'seqname': 'chrom'})
)

def process_one_file(fname):
    import os
    import pandas as pd
    from pybedtools import BedTool

    pqtl_file = os.path.join(pqtl_dir, fname)
    sig_rows = []

    try:
        # Read in chunks
        for chunk in pd.read_csv(pqtl_file, sep='\t', compression='gzip', chunksize=500_000):
            filtered = chunk[chunk['Pval'].astype(float) < 5e-8]
            if not filtered.empty:
                sig_rows.append(filtered)
    except Exception as e:
        print(f"Could not read {fname}: {e}")
        return

    if not sig_rows:
        return

    # Concatenate all significant rows
    pqtl_df = pd.concat(sig_rows, ignore_index=True)
    # Prepare BED for overlap
    pqtl_bed = BedTool.from_dataframe(
        pqtl_df.assign(
            chrom=pqtl_df['Chrom'],
            start=pqtl_df['Pos'] - 1,
            end=pqtl_df['Pos']
        )[['chrom', 'start', 'end', 'Name']]
    )
    overlap = pqtl_bed.intersect(dhs_bed, wa=True, u=True)
    overlap_names = set([x.name for x in overlap])
    pqtl_filtered = pqtl_df[pqtl_df['Name'].isin(overlap_names)]
    if not pqtl_filtered.empty:
        basename = fname.replace('.txt.gz', '_dhs.txt.gz').replace('.txt', '_dhs.txt.gz')
        pqtl_filtered.to_csv(os.path.join(output_dir, basename), sep='\t', index=False, compression='gzip')


if __name__ == '__main__':
    # files = [f for f in os.listdir(pqtl_dir) if f.endswith('.txt.gz')]
    files = [ f for f in qtlfiles if f.endswith( '.txt.gz' ) ]
    max_workers = 6  # or os.cpu_count(), adjust for your machine
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        list(executor.map(process_one_file, files))
    print("All done!")


/tmp/ipykernel_68782/3318579270.py:13: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dhs = pd.read_csv(dhs_path, sep='\t')


Could not read 13228_75_MDM2_MDM2.txt.gz: Error -3 while decompressing data: invalid stored block lengths
Could not read 13231_90_IGHG4_IgG4__Kappa.txt.gz: Error -3 while decompressing data: invalid block type
Could not read 13236_25_WNT3A_WNT3A.txt.gz: Error -3 while decompressing data: invalid stored block lengths
Could not read 13229_20_MDM4_MDM4.txt.gz: Error tokenizing data. C error: Expected 12 fields in line 29316164, saw 30

Could not read 13242_134_SHANK3_SHAN3.txt.gz: Error tokenizing data. C error: Expected 12 fields in line 8553054, saw 14

Could not read 13240_170_SHANK1_SHAN1.txt.gz: Error tokenizing data. C error: Expected 12 fields in line 10977209, saw 15

Could not read 13230_174_IGHG2_IgG2__Kappa.txt.gz: Error -3 while decompressing data: invalid distance code
Could not read 13526_5_SSB_LA.txt.gz: Compressed file ended before the end-of-stream marker was reached
Could not read 13535_2_COL4A3BP_C43BP.txt.gz: Compressed file ended before the end-of-stream marker was re

**Step 5:** Load tabular data (summary stats / annotations).

In [7]:


# %% deCODE pQTL → MR exposure (within DHS; one SNP per gene×DHS)
import os, glob, re
from pathlib import Path
import pandas as pd
import numpy as np

# ---------- Inputs ----------
input_dir = "/mnt/f/10_osteo_MR/datasets/dhs_context/pqtl_decode/"   # per-gene files like: 9999_1_IRF6_IRF6_dhs.txt.gz
outcome_with_dhs = "/mnt/f/10_osteo_MR/MR_ready/outcome_osteo_within_wb_DHS.tsv"  # columns: SNP (may be 'rs1;rs2;...'), dhs_id

# ---------- Output ----------
output_file = "/mnt/f/10_osteo_MR/MR_ready/exposure_pqtl_decode_dhs_index.tsv"
Path(os.path.dirname(output_file)).mkdir(parents=True, exist_ok=True)

# ---------- 0) RSIDs within DHS (and dhs_id) from outcome ----------
splitter = re.compile(r"[;,\s]+")
snp2dhs = {}
out_df = pd.read_csv(outcome_with_dhs, sep="\t", dtype="string", usecols=["SNP","dhs_id"])
for s, did in zip(out_df["SNP"], out_df["dhs_id"]):
    if pd.isna(s): 
        continue
    did_str = "" if pd.isna(did) else str(did)
    for tok in splitter.split(str(s)):
        if tok and tok != "." and tok not in snp2dhs:
            snp2dhs[tok] = did_str
rsids_in_dhs = set(snp2dhs.keys())
print(f"[info] outcome DHS RSIDs: {len(rsids_in_dhs):,}")

# ---------- 1) Load & merge all deCODE per-gene pQTL files ----------
files = sorted(set(glob.glob(os.path.join(input_dir, "*_dhs.txt.gz"))) |
               set(glob.glob(os.path.join(input_dir, "*_dhs.txt.gz.gz"))))
if not files:
    raise FileNotFoundError(f"No *_dhs.txt.gz(.gz) files found in {input_dir}")

def infer_gene_from_filename(fpath: str) -> str:
    # Example: "9999_1_IRF6_IRF6_dhs.txt.gz" -> "IRF6" (use token before 'dhs')
    base = os.path.basename(fpath)
    toks = base.split("_")
    if "dhs" in toks:
        idx = toks.index("dhs")
    # fallback: the 3rd token if it exists
    return toks[2] if len(toks) >= 3 else toks[0]

def pick_snp_column(df: pd.DataFrame) -> pd.Series:
    # deCODE files sometimes use 'rsids' or 'Name'
    if "rsids" in df.columns: return df["rsids"].astype("string")
    if "Name"  in df.columns: return df["Name"].astype("string")
    if "rsid"  in df.columns: return df["rsid"].astype("string")
    raise ValueError("SNP column not found (expected one of: rsids, Name, rsid)")

def pick_col(df, *names, default=None):
    for n in names:
        if n in df.columns:
            return df[n]
    if default is not None:
        return default
    raise ValueError(f"None of the columns found: {names}")

records = []
for f in files:
    try:
        df = pd.read_csv(f, sep="\t", compression="infer", dtype="string")
        gene = infer_gene_from_filename(f)

        # Choose/normalize essential columns
        snp_raw = pick_snp_column(df)                                 # rsids/Name/rsid
        ea     = pick_col(df, "effectAllele","ALLELE1", default=pd.Series([pd.NA]*len(df)))
        oa     = pick_col(df, "otherAllele","ALLELE0", default=pd.Series([pd.NA]*len(df)))
        beta   = pick_col(df, "Beta","BETA", default=pd.Series([pd.NA]*len(df)))
        se     = pick_col(df, "SE", default=pd.Series([pd.NA]*len(df)))
        pval   = pick_col(df, "Pval", default=None)
        if pval is None and "LOG10P" in df.columns:
            pval = (10.0 ** (-pd.to_numeric(df["LOG10P"], errors="coerce"))).astype("float")
        else:
            pval = pd.to_numeric(pval, errors="coerce")

        tmp = pd.DataFrame({
            "snp_raw": snp_raw,
            "effect_allele": ea,
            "other_allele": oa,
            "beta": pd.to_numeric(beta, errors="coerce"),
            "se":   pd.to_numeric(se, errors="coerce"),
            "pval": pval,
            "gene": gene
        })

        # Some files may carry a DHS identifier column; keep if present
        if "DHS_identifier" in df.columns:
            tmp["dhs_id_from_file"] = df["DHS_identifier"].astype("string")

        records.append(tmp)
    except Exception as e:
        print(f"[warn] Skipping {f}: {e}")

merged = pd.concat(records, ignore_index=True) if records else pd.DataFrame(columns=[
    "snp_raw","effect_allele","other_allele","beta","se","pval","gene","dhs_id_from_file"
])
print(f"[merge] total rows read: {len(merged):,}")

# ---------- 2) Normalize SNPs & filter to DHS RSIDs from outcome ----------
# If snp_raw contains multiple rsids (e.g., "rs1;rs2"), keep the first that appears in rsids_in_dhs
def first_rsid_in_set(s: str) -> str:
    if s is None or pd.isna(s): return ""
    toks = [t for t in splitter.split(str(s)) if t]
    for t in toks:
        if t in rsids_in_dhs:
            return t
    return ""  # no acceptable token

merged["SNP"] = merged["snp_raw"].map(first_rsid_in_set)
merged = merged[(merged["SNP"] != "")].copy()
print(f"[filter] rows with SNP in outcome DHS set: {len(merged):,}")

# Attach dhs_id from outcome mapping
merged["dhs_id"] = merged["SNP"].map(snp2dhs).astype("string")

# ---------- 3) Compute |Z| and select one SNP per (gene × dhs_id) ----------
merged["Zabs"] = (merged["beta"] / merged["se"]).abs()
merged.loc[~np.isfinite(merged["Zabs"]), "Zabs"] = np.nan

# Drop rows missing key fields before ranking
merged = merged.dropna(subset=["pval","beta","se","effect_allele","other_allele","dhs_id","gene"])

# Rank: lowest pval, then largest |Z|, then first
merged = merged.sort_values(["gene","dhs_id","pval","Zabs"], ascending=[True,True,True,False])
best = merged.groupby(["gene","dhs_id"], as_index=False).head(1).copy()
print(f"[select] final (gene × DHS) picks: {len(best):,}")

# ---------- 4) Build MR-ready output ----------
mr = best[["SNP","effect_allele","other_allele","beta","se","pval","gene","dhs_id"]].copy()

mr.to_csv(output_file, sep="\t", index=False)
print(f"[done] MR exposure (deCODE pQTL) written: {output_file}  (rows: {len(mr):,})")


[info] outcome DHS RSIDs: 606,769
['13632', '10', 'ZYX', 'Zyxin', 'dhs', 'dhs.txt.gz.gz']
[merge] total rows read: 775,999
[filter] rows with SNP in outcome DHS set: 435,693
[select] final (gene × DHS) picks: 243,652
[done] MR exposure (deCODE pQTL) written: /mnt/f/10_osteo_MR/MR_ready/exposure_pqtl_decode_dhs_index.tsv  (rows: 243,652)
